# 背景：使用autogluon

## 导包、设置根目录

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import random
import pickle
import multiprocessing
import re
import pickle

from collections import Counter
from itertools import chain
import datetime

from autogluon.tabular import TabularDataset,TabularPredictor

# 更改工作目录为当前项目根目录
import sys
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.chdir(os.path.dirname(os.path.dirname(sys.path[0])))
print(os.getcwd())

/Users/jincan02/Projects/Log-diagnosis


## 读取日志和标签数据

In [2]:
word_df = pd.read_csv('./pre_contest/v1p2/word_label_df.txt',sep='\t',index_col=0)
log_list=list(word_df['log'])
label_list=list(word_df['label'])
word_df.groupby('label').size()

label
0    1460
1    3016
2    7731
3    2214
dtype: int64

## 读取v1p2特征重要性前300的特征作为本次使用的数据集

In [3]:
feature_importance_top_300_df=pd.read_csv('./pre_contest/v1p2/feature_importance_top_300_df.csv',sep=',')

In [4]:
feature_importance_top_300_df.head()

,sn,day,label,memory,or,cpu,processor,caterr,configuration,e,...,36,check,0xe1,f1,cycle,disk10,0xec,button,dimmg1_temp,cpu1_c1d1
0,SERVER_10001,2020-05-01,1,0,9,4,4,0,0,60,...,0,0,0,0,0,0,0,0,0,0
1,SERVER_10003,2020-03-28,2,1,2,1,0,0,0,6,...,0,0,0,0,0,0,0,0,0,0
2,SERVER_10008,2020-02-25,1,1,5,2,2,0,1,14,...,0,0,0,0,0,0,0,0,0,0
3,SERVER_10008,2020-03-11,2,1,5,2,2,0,1,20,...,0,0,0,0,0,0,0,0,0,0
4,SERVER_10009,2020-05-08,3,0,0,0,0,0,0,18,...,0,0,0,0,0,0,0,0,0,0


In [5]:
feature_names_list=list(feature_importance_top_300_df.columns)[3:33]
feature_df=feature_importance_top_300_df[feature_names_list]
label_list=list(feature_importance_top_300_df['label'])

## 划分训练集和验证集

In [6]:
random.seed(0)
val_mask = [random.random() < 0.3 for _ in range(len(feature_df))]
train_mask = [not xx for xx in val_mask]
train_data=feature_importance_top_300_df[['label']+feature_names_list][train_mask]
label=np.array(label_list)
val_feature = feature_df[val_mask]
val_label = label[val_mask]
train_feature = feature_df[train_mask]
train_label = label[train_mask]

## 使用autogluon训练模型

In [ ]:
save_path='pre_contest/v1p3/autogluon'
predictor=TabularPredictor(label='label',eval_metric='f1_macro',path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "pre_contest/v1p3/autogluon/"
AutoGluon Version:  0.4.0
Python Version:     3.8.12
Operating System:   Darwin
Train Data Rows:    10133
Train Data Columns: 30
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	4 unique label values:  [1, 2, 3, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 4
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5730.55 MB
	Train Data (Original)  Memory Usage: 2.43 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dt

In [ ]:
# 指标评估
def macro_f1(label,prediction)  -> float:

    """
    计算得分
    :param target_df: [sn,fault_time,label]
    :param submit_df: [sn,fault_time,label]
    :return:
    """

    weights =  [3  /  7,  2  /  7,  1  /  7,  1  /  7]
    macro_F1 =  0.
    for i in  range(len(weights)):
        TP =  np.sum((label==i) & (prediction==i))
        FP =  np.sum((label!= i) & (prediction == i))
        FN =  np.sum((label == i) & (prediction!= i))
        precision = TP /  (TP + FP)  if  (TP + FP)  >  0  else  0
        recall = TP /  (TP + FN)  if  (TP + FN)  >  0  else  0
        F1 =  2  * precision * recall /  (precision + recall)  if  (precision + recall)  >  0  else  0
        macro_F1 += weights[i]  * F1
        
        print('Task %d:\n Prcesion %.2f, Recall %.2f, F1 %.2f' % (i+1, precision, recall, F1))
        
    return macro_F1

In [ ]:
macro_f1(train_label,train_pred)

In [ ]:
macro_f1(val_label,val_pred)
# 采用所有词的词频向量比采用词频+word2vec效果更佳